# Data Cleaning and Preparation

This is probably 80% of a data analysts job.

In [3]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

In pandas objects, missing data is represented somewhat imperfect but functional for many users. For numeric data, the floating-point value `NaN` (Not a Number) is used, a _sentinel value_ that can be easily detected.

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

For other data types, missing data is referred to as `NA`, _not available_. In statistics applications, NA data may either not exist or may exist but was not observed. It is often important to do analysis on the missing data to identify data collection problems or potential biases in the data caused by missing data.

The built-in Python `None` value is also treated as NA in object arrays:

In [5]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

_Table: NA handling methods_

| Argument | Description
| :--- | :---- |
| dropna | Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate.
| fillna | Fill in missing data with some value or using an interpolation method such as 'ffill' (forward-filling) or 'bfill' (backwards-filling).
| isnull | Return boolean values indicating which values are missing/NA.
| notnull | Negation ofisnull.

### Filtering Out Missing Data

* `pandas.isnull` and boolean indexing is always an option
* `dropna` can be helpful, too

`dropna` on a Series returns the Series with only the non-null data and index values:

In [6]:
from numpy import nan as NA

In [9]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [8]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is equivalent to:

In [10]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

It is more complex with DataFrame objects as you may want to drop rows or columns that are all NA or only those containing any NAs.
`dropna` drops by default any _row_ containing a missing value:

In [11]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [12]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing `how='all'` will only drop rows that are all NA:

In [13]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Pass `axis=1` to drop columns in the same way:

In [14]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [15]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


With time series data (or other types of data, e.g. genetic data), one might want to keep only rows containing a certain number of observations. Use the `thresh` argument to achieve that:

In [16]:
df = pd.DataFrame(np.random.randn(7, 3))

In [17]:
df.iloc[:4, 1] = NA

In [18]:
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [19]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [20]:
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### Filling In Missing Data

It can be useful to fill in missing data with some value, instead of filtering out missing data and discarding other data along with it. Calling `fillna` with a constant replaces missing values with that value: 

In [21]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


Calling `fillna` with a dict, you can use a different fill value for each column:

In [22]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


`fillna` returns a new object, but you can modify the existing object in-place:

In [23]:
df.fillna(0, inplace=True)
df

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


The same interpolation methods available for reindexing can be used:

In [24]:
df = pd.DataFrame(np.random.randn(6, 3))

In [25]:
df.iloc[2:, 1] = NA

In [26]:
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


Use the forward-filling method to replace missing data. `'ffill'` will take the last available value to fill in missing data.

In [27]:
df.fillna(method='ffill')

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,0.124121,-2.370232
5,-1.265934,0.124121,-2.370232


In [28]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


With a little creativity, you can do many other things with `fillna`. For example, you might pass the mean or median value of a Series:

In [29]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [30]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

_Table: fillna function arguments_

| Argument | Description
| :--- | :---- |
| value | Scalar value or dict-like object to use to fill missing values..
| method | Interpolation; by default 'ffill' if function called with no other arguments.
| axis | Axis to fill on; default axis=0.
| inplace | Modify the calling object without producing a copy.
| limit | For forward and backward filling, maximum number of consecutive periods to fill.

## Data Transformation

### Removing Duplicates

Example for duplicate rows:

In [31]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method `duplicated` returns a boolean Series indicating whether each row is a duplicate (=has been observed in a previous row) or not:

In [32]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicates` returns a DataFrame where the duplicated array is `False`:

In [33]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Default is to consider all columns. It is possible to specify a subset of columns:

In [39]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [35]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


The first observed value combination is kept by default. Use `keep='last'` to return the last one:

In [40]:
data.drop_duplicates(['k1', 'k2'], keep='last') # This does not make a difference. Bug?

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

Transformation based on the values in an array, Series or DataFrame column.

In [41]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Add a column with the type of animal that each food came from

In [42]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

First, we need to convert all values to lowercase:

In [43]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

The `map` method on a Series accepts a function of dict-like object containing a mapping.

In [44]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


Alternatively, we could have passed a function:

In [45]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values

`fillna` is a special case of a more general value replacement. `replace` provides a simple and flexible way to modify a subset of values in an object:

In [4]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

Replace `-999.0` values with `NaN` (pandas sentinel value for missing data):

In [5]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

To replace multiple values at once, pass a list and then the substitute value:

In [48]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:

In [49]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

It is also possible to pass a dict:

In [50]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The `data.replace` method is distinct from `data.str.replace` which performs string substitution element-wise (see further down).

### Renaming Axis Indexes

Axis labels can be transformed by a function of mapping of some sort, either to produce new, differently labeled objects or in-place.

In [6]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Like a Series, the axis indexes have a `map` method:

In [52]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

Assign to `index`, modifying the DataFrame in-place:

In [53]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Use `rename` to create a transformed version of a dataset without modifying the original:

In [54]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In conjunction with a dict-like object, `rename` can provide new values for a subset of the axis labels:

In [55]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Modify a dataset in-place by passing `inplace=True`:

In [56]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

Continuous data is often discretized or otherwise separated into bins for analysis

In [3]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Divide them into bins of 18-25, 26-35, 36-60 and 61 and older with `cut`:

In [4]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

* the returned pandas object is a special `Categorical object`
* the output describes the bins computed by `cut`
* it can be treated like an array of strings indicating the bin name
* internally it contains a `categories` array specifying the distinct category names along with a labeling for the `ages` data in the `codes` attribute.

In [5]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [6]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [7]:
pd.value_counts(cats) # bin counts for the result of pandas.cut

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

Parenthesis mean that the given side is _open_ while square brackets mean it is _closed_ (inclusive), consistent with mathematical notation for intervals. You can change which side is closed by passing `right=False`:

In [8]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

Pass your own bin names by passing a list or array to the `labels` option:

In [9]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

By passing an integer number of bins to `cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data.

In [7]:
data = np.random.rand(20)
data

array([ 0.9296,  0.3164,  0.1839,  0.2046,  0.5677,  0.5955,  0.9645,
        0.6532,  0.7489,  0.6536,  0.7477,  0.9613,  0.0084,  0.1064,
        0.2987,  0.6564,  0.8098,  0.8722,  0.9646,  0.7237])

In [10]:
cats = pd.cut(data, 4, precision=2)
cats

[(0.73, 0.96], (0.25, 0.49], (0.0074, 0.25], (0.0074, 0.25], (0.49, 0.73], ..., (0.49, 0.73], (0.73, 0.96], (0.73, 0.96], (0.73, 0.96], (0.49, 0.73]]
Length: 20
Categories (4, interval[float64]): [(0.0074, 0.25] < (0.25, 0.49] < (0.49, 0.73] < (0.73, 0.96]]

`precision=2` limits the decimal precision to two digits.

In [11]:
pd.value_counts(cats) # bin counts for the result of pandas.cut

(0.73, 0.96]      8
(0.49, 0.73]      6
(0.0074, 0.25]    4
(0.25, 0.49]      2
dtype: int64

`qcut` bins the data based on sample quantiles. Depending on the data distribution, `cut` will not usually result in equal numbers per bin. By using sample quantiles, by definition `qcut` results in roughly equal-size bins:

In [11]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(0.626, 3.928], (0.626, 3.928], (-2.95, -0.691], (-0.691, -0.0171], (0.626, 3.928], ..., (-0.0171, 0.626], (0.626, 3.928], (-0.691, -0.0171], (-0.691, -0.0171], (-0.0171, 0.626]]
Length: 1000
Categories (4, interval[float64]): [(-2.95, -0.691] < (-0.691, -0.0171] < (-0.0171, 0.626] < (0.626, 3.928]]

In [12]:
pd.value_counts(cats)

(0.626, 3.928]       250
(-0.0171, 0.626]     250
(-0.691, -0.0171]    250
(-2.95, -0.691]      250
dtype: int64

You can pass your own quantiles instead (numbers between 0 and 1, inclusive):

In [13]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(1.297, 3.928], (-0.0171, 1.297], (-2.95, -1.191], (-1.191, -0.0171], (1.297, 3.928], ..., (-0.0171, 1.297], (-0.0171, 1.297], (-1.191, -0.0171], (-1.191, -0.0171], (-0.0171, 1.297]]
Length: 1000
Categories (4, interval[float64]): [(-2.95, -1.191] < (-1.191, -0.0171] < (-0.0171, 1.297] < (1.297, 3.928]]

### Detecting and Filtering Outliers

Filtering or transforming outliers is often a matter of applying array operations.

In [14]:
data = pd.DataFrame(np.random.randn(1000, 4)) # normally distributed data
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.021425,-0.005165,-0.055670,0.044107
std,1.008404,0.995532,0.994484,0.996884
min,-3.184377,-3.745356,-3.428254,-3.645860
25%,-0.628122,-0.699383,-0.747478,-0.599807
50%,-0.016127,-0.029924,-0.091364,0.043663
75%,0.690847,0.694459,0.620197,0.740562
max,3.525865,2.735527,3.366626,2.653656


Find values in one of the columns exceeding 3 in absolute value:

In [15]:
col = data[2]
col[np.abs(col) > 3]

269   -3.428254
646    3.366626
Name: 2, dtype: float64

Use the `any` method on a boolean DataFrame to select all rows having a value exceeding 3 or -3:

In [16]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
52,-0.025907,-3.399312,-0.974657,-0.685312
71,3.260383,0.963301,1.201206,-1.852001
147,-0.196713,-3.745356,-1.520113,-0.346839
246,-3.056990,1.918403,-0.578828,1.847446
269,0.326045,0.425384,-3.428254,-0.296336
333,-3.184377,1.369891,-1.074833,-0.089937
554,0.208011,-0.150923,-0.362528,-3.548824
646,0.193299,1.397822,3.366626,-2.372214
793,3.525865,0.283070,0.544635,0.462204
813,-0.450721,-0.080332,0.599947,-3.645860


Values can be set based on these criteria. Cap values outside the interval -3 to 3:

In [17]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.020880,-0.004021,-0.055608,0.045301
std,1.005160,0.991718,0.991924,0.992863
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.628122,-0.699383,-0.747478,-0.599807
50%,-0.016127,-0.029924,-0.091364,0.043663
75%,0.690847,0.694459,0.620197,0.740562
max,3.000000,2.735527,3.000000,2.653656


The statement `np.sign(data)` produces 1 and -1 values based on whether the values in `data` are positive or negative:

In [18]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,1.0
1,-1.0,-1.0,-1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,1.0,1.0,-1.0,-1.0
4,1.0,-1.0,-1.0,-1.0


### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows of a DataFrame is easy with `numpy.random.permutation`. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [19]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([1, 0, 4, 2, 3])

That array can then be used in `iloc`-based indexing or the equivalent `take` function:

In [21]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [22]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19
2,8,9,10,11
3,12,13,14,15


To select a random subset without replacement, you can use `sample` on Series and DataFrame:

In [23]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
1,4,5,6,7


To generate a sample _with_ replacement (to allow repeat choices), pass `replace=True` to `sample`:

In [24]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

0    5
3    6
1    7
4    4
0    5
3    6
0    5
3    6
2   -1
3    6
dtype: int64

### Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a "dummy" or "indicator" matrix. If a column in a DataFrame has `k` distinct values, you would derive a matrix or DataFrame with `k` columns containing all 1s and 0s. pandas has a `get_dummies` function for this, though devising one yourself is not difficult. One example could be a column with the gender, and replacing male and female with 0s and 1s.

In [26]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [25]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data. `get_dummies`has a prefix argument for doing this.

In [27]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


If a row in a DataFrame belongs to multiple categories, things are a bit more complicated, e.g. in the MovieLens 1M dataset:

In [13]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

/Users/verenakutschera/miniconda3/envs/pydata-book/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Some wrangling is required to add indicator variables for each genre. First, extract the list of unique genres in the dataset:

In [14]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [15]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

One way to construct the indicator DataFrame is to start with a DataFrame of all zeros:

In [16]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now, iterate through each movie and set entries in each row of `dummies` to 1. To do thie, we use the `dummies.columns` to compute the column indices for each genre:

In [17]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [18]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

Then, we can use `.iloc` to set values based on these indices:

In [19]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

Then you can combine this with movies:

In [20]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
                                ...             
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

For much larger data, this method is not very fast. Instead, you can write a lower-level function that writes directly to a NumPy array, and then wrap the result in a DataFrame.

A useful recipe for statistical applications is to combine `get_dummies` with a discretization function like `cut`:

In [38]:
np.random.seed(12345) # set the random seed to make the example deterministic
values = np.random.rand(10)
values

array([ 0.9296,  0.3164,  0.1839,  0.2046,  0.5677,  0.5955,  0.9645,
        0.6532,  0.7489,  0.6536])

In [37]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation

pandas adds to Python's string objects' methods and regular expressions by enabling their application on whole arrays of data, additionally handling missing data

### String Object Methods

Example for built-in string method:

In [39]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

`split` is often combined with `strip` to trim whitespace (incl. line breaks):

In [40]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

Concatenate substrings with a two-colon delimiter:

In [41]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

Faster and more pythonic way: pass a list or tuple to the `join` method on the string `'::'`:

In [42]:
'::'.join(pieces)

'a::b::guido'

Locate substrings using Python's `in` keyword to find a substring, alternatively, use `index` and `find`:

In [43]:
'guido' in val

True

In [44]:
val.index(',')

1

In [45]:
val.find(':')

-1

`index` raises and exception if the string isn't found (versus returning -1):

In [46]:
val.index(':')

ValueError: substring not found

`count` returns the number of occurrences of a particular substring

In [47]:
val.count(',')

2

`replace` substitutes occurrences of one pattern for another. Delete patterns by passing an empty string:

In [48]:
val.replace(',', '::')

'a::b::  guido'

In [49]:
val.replace(',', '')

'ab  guido'

_Table: Python built-in string methods_

| Argument | Description
| :--- | :---- |
| count | Return the number of non-overlapping occurrences of substring in the string. 
| endswith | Returns `True` if string ends with suffix.
| startswith | Returns `True` if string starts with prefix.
| join | Use string as delimiter for concatenating a sequence of other strings.
| index | Return position of first character in substring if found in the string; raises `ValueError` if not found.
| find | Return position of first character of first occurrence of substring in the string; like index, but returns –1 if not found.
| rfind | Return position of first character of last occurrence of substring in the string; returns –1 if not found.
| replace | Replace occurrences of string with another string.
| strip, rstrip, lstrip | Trim whitespace, including newlines; equivalent to x.strip() (and rstrip, lstrip, respectively) for each element.
| split | Break string into list of substrings using passed delimiter.
| lower | Convert alphabet characters to lowercase.
| upper | Convert alphabet characters to uppercase.
| casefold | Convert characters to lowercase, and convert any region-specific variable character combinations to a common comparable form.
| ljust, rjust | Left justify or right justify, respectively; pad opposite side of string with spaces (or some other fill character) to return a string with a minimum width.

### Regular Expressions

Search or match (complex) string patterns in text. A single expression (a _regex_) is a string formed according to the regular expression language. Python has a built-in `re` module for that. `re` module functions fall into three categories of functions: pattern matching, substitution, splitting.

Split a string with a variable number of whitespace characters (tabs, spaces, newlines). The regex for one or more whitespace characters is `\s+`.

In [50]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

When calling `re.split()`, the regular expression is first compiled, and then its `split` method is called on the passed text. You can compile the regex yourself with `re.compile`, forming a reusable regex object:

In [51]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

To get a list of all patterns matching the regex, use `findall`:

In [52]:
regex.findall(text)

['    ', '\t ', '  \t']

To avoid unwanted escaping with \ in a regular expression, use _raw_ string literals like `r'C:\x'` instead of the equivalent `'C:\\x'`.

If you intend to apply the same expression to many strings, it is highly recommended to create a regex object with `re.compile` to save CPU.

`match` and `search` are closely related to `findall`. 
* `search` returns the first match. 
* `match` only matches at the beginning of the string.

Consider a block of text and a regular expression to identify most email addresses:

In [53]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

Using `findall` on the text produces a list of the email addresses:

In [54]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`search` returns a special match object for the first email address in the text. for the preceding regex, the match object can only tell us the start and end position of the pattern in the string:

In [55]:
m = regex.search(text)
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [56]:
text[m.start():m.end()]

'dave@google.com'

`regex.match` returns `None`, because it only matches if the pattern occurs at the start of the string:

In [57]:
print(regex.match(text))

None


`sub` will return a new string with occurrences of the pattern replaced by a new string:

In [58]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



To find email addresses and simultaneously segment each address into its three components (username, domain name, domain suffix), put parentheses around the parts of the pattern to segment:

In [59]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

A match object produced with this regex returns a tuple of the pattern components with its `groups` method:

In [60]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

`findall` returns a list of tuples when the pattern has groups:

In [61]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

`sub` also has access to groups in each match using special symbols like \1 and \2. \1 corresponds to the first matched group, \2 to the second, and so forth:

In [62]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



_Table: Regular expression methods_

| Argument | Description
| :--- | :---- |
| findall | Return all non-overlapping matching patterns in a string as a list.
| finditer | Like findall, but returns an iterator.
| match | Match pattern at start of string and optionally segment pattern components into groups; if the pattern matches, returns a match object, and otherwise None.
| search | Scan string for match to pattern; returning a match object if so; unlike match, the match can be anywhere in the string as opposed to only at the beginning.
| split | Break string into pieces at each occurrence of pattern.
| sub, subn | Replace all (sub) or first n occurrences (subn) of pattern in string with replacement expression; use symbols \1, \2, ... to refer to match group elements in the replacement string.  

### Vectorized String Functions in pandas

Cleaning up a messy dataset for analysis often requires a lot of string munging and regularization. To complicate things, a column containing strings will sometimes have missing data.
* vectorized = functions are applied per row

In [63]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [64]:
data.isnull()

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

String and regular expression methods can be applied (with `lambda` or other functions) to each value using `data.map`, but it will fail on the NA (null) values. Series has array-oriented methods for string operations that skip NA values, through the `str` attribute.

In [65]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

Regular expressions can be used, too, along with any `re` options like `IGNORECASE`:

In [66]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [67]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

There are a couple of ways to do vectorized element retrieval:
* use `str.get`
* index into the `str` attribute

In [68]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Rob      True
Steve    True
Wes       NaN
dtype: object

To access elements in the embedded lists, we can pass an index to either of these functions:

In [69]:
matches.str.get(1)

Dave    NaN
Rob     NaN
Steve   NaN
Wes     NaN
dtype: float64

In [70]:
matches.str[0]

Dave    NaN
Rob     NaN
Steve   NaN
Wes     NaN
dtype: float64

Slice strings using this syntax:

In [71]:
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object

_Table: Partial listing of vectorized string methods_

| Method | Description
| :--- | :---
| cat | Concatenate strings element-wise with optional delimiter.
| contains | Return boolean array if each string contains pattern/regex.
| count | Count occurrences of pattern.
| extract | Use a regular expression with groups to extract one or more strings from a Series of strings; the result will be a DataFrame with one column per group.
| endswith | Equivalent to `x.endswith(pattern)` for each element.
| startswith | Equivalent to `x.startswith(pattern)` for each element.
| findall | Compute list of all occurrences of pattern/regex for each string.
| get | Index into each element (retrieve i-th element).
| isalnum | Equivalent to built-in `str.alnum`.
| isalpha | Equivalent to built-in `str.isalpha`.
| isdecimal | Equivalent to built-in `str.isdecimal`.
| isdigit | Equivalent to built-in `str.isdigit`.
| islower | Equivalent to built-in `str.islower`.
| isnumeric | Equivalent to built-in `str.isnumeric`.
| isupper | Equivalent to built-in `str.isupper`.
| join | Join strings in each element of the Series with passed separator
| len | Compute length of each string 
| lower,upper | Convertcases; equivalent to `x.lower()` or `x.upper()` for each element.
| match | Use re.match with the passed regular expression on each element, returning matched groups as list.
| pad | Add whitespace to left, right, or both sides of strings.
| center | Equivalent to `pad(side='both')`.
| repeat | Duplicate values (e.g., `s.strrepeat(3)` is equivalent to x * 3 for each string).
| replace | Replace occurrences of pattern/regex with some other string.
| slice | Slice each string in the Series.
| split | Split strings on delimiter or regular expression.
| strip | Trim whitespace from both sides, including newlines.
| rstrip | Trim whitespace on right side.
| lstrip | Trim whitespace on left side.